<a href="https://colab.research.google.com/github/rathodr1526/Ranjit-rathod/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!pip install flask
!pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00


In [9]:
import spacy
import flask
from flask import  Flask , request , jsonify
from transformers import Conversation , pipeline
import PyPDF2

In [7]:
#Initialize NLP pipelines
nlp = spacy.load('en_core_web_sm')

In [8]:
# Initialize chat pipelines
chat_pipeline = pipeline('conversational')

No model was supplied, defaulted to microsoft/DialoGPT-medium and revision 8bada3b (https://huggingface.co/microsoft/DialoGPT-medium).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
# Initialize the question-answering pipeline
qa_pipeline = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
#Inialize flask app
app = Flask('swadhyay')

In [11]:
#get text from pdf 
def extract_text_from_pdf(file):
  pdf_reader = PyPDF2.Pdfreader(file)
  text = ''
  for page in pdf_reader.pages:
    text += page.extract_text()
  return text

In [16]:

def chatbot():

  file = request.files['file']
  user_message = request.form['message']


  #extract text from PDF file
  context = extract_text_from_pdf(file)


  # perform named entity recognition using Spacy
  doc = nlp(user_message)
  entities = []
  for ent in doc.ents:
    entities.append({'text':ent.text , 'label' : ent.label_})


  #craete conversational history
  conversation = Conversation()
  conversation.add_user_input(user_message)

  #Append conversation history with previous chat messeges
  chat_history = conversation.get_history()

  #generate chat response
  response = chat_pipeline(chat_history)
  bot_response = response.generated_responses[-1]

  #perform question-answering using transformer pipeline
  answer = qa_pipeline({
      
                        'question' : user_messeges,
                        'context' : context
  })

  response = {
      'entities' : entities,
      'answer' : answer['answer']
  }

  return jsonify(response)

  if __name__ == '__main__':
    app.run()
